In [ ]:
import json
import os
import time
import base64
from google import genai
from google.genai import types
from tqdm.notebook import tqdm
from PIL import Image

# --- CONFIGURATION ---
CONFIG = {
    "skip_existing": True,      # Skip files that already exist
    "dry_run": False,           # Preview without API calls
    "categories": ["projects", "meta_upgrades", "upgrades", "jobs"],  # What to generate
    "rate_limit_delay": 2,      # Delay between API calls (seconds)
    "max_retries": 3,           # Max retry attempts per image
    "validate_images": True     # Validate generated images
}

PROJECT_ID = "bobenddk-personal"
LOCATION = "global"
OUTPUT_DIR = "images"

# Statistics tracking
stats = {
    "total": 0,
    "success": 0,
    "skipped": 0,
    "failed": 0,
    "failed_items": []
}

# --- STYLE PROMPTS ---
STYLE_ICON = (
    "Windows 95 aesthetic pixel art icon. "
    "16-bit graphics, 256 color VGA palette, dithered shading, low resolution, blocky. No extra Text. No borders. "
    "Must be readable in small icon sizes. "
    "Subject: "
)

STYLE_SPLASH = (
    "Wide horizontal pixel art illustration, Windows 95 splash screen style. "
    "Retro computing interface, 256 color VGA palette, dithered textures, low resolution CRT monitor effect. No extra Text. No borders. "
    "Scene: "
)

# ----------------------------------------------

def init_ai():
    print(f"Initializing Google Gen AI Client in {LOCATION}...")
    client = genai.Client(
        vertexai=True,
        project=PROJECT_ID,
        location=LOCATION
    )
    return client

def validate_image(filename):
    """Validate that the generated image is valid"""
    try:
        img = Image.open(filename)
        img.verify()
        # Check file size (should be > 1KB)
        if os.path.getsize(filename) < 1024:
            return False
        return True
    except Exception as e:
        print(f"  Image validation failed: {e}")
        return False

def generate_image(client, full_prompt, output_filename, aspect_ratio="1:1"):
    """Generate a single image"""
    prompt_text = f"{full_prompt} Aspect Ratio: {aspect_ratio}"

    try:
        response = client.models.generate_content(
            model="gemini-3-pro-image-preview",
            contents=[
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_text(text=prompt_text)
                    ]
                )
            ],
            config=types.GenerateContentConfig(
                temperature=1,
                response_modalities=["IMAGE"],
                safety_settings=[
                    types.SafetySetting(
                        category="HARM_CATEGORY_HATE_SPEECH",
                        threshold="OFF"
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_DANGEROUS_CONTENT",
                        threshold="OFF"
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
                        threshold="OFF"
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_HARASSMENT",
                        threshold="OFF"
                    )
                ]
            )
        )

        # Extract and save image
        if response.candidates:
            for part in response.candidates[0].content.parts:
                if part.inline_data:
                    data = part.inline_data.data
                    if isinstance(data, str):
                        img_data = base64.b64decode(data)
                    else:
                        img_data = data

                    with open(output_filename, "wb") as f:
                        f.write(img_data)
                    
                    # Validate image if enabled
                    if CONFIG["validate_images"]:
                        if validate_image(output_filename):
                            return True
                        else:
                            print(f"  Generated image failed validation")
                            os.remove(output_filename)
                            return False
                    return True

        print(f"  No image found in response")
        return False

    except Exception as e:
        raise Exception(f"Generation error: {e}")

def generate_image_with_retry(client, full_prompt, output_filename, aspect_ratio="1:1"):
    """Generate image with retry logic and exponential backoff"""
    for attempt in range(CONFIG["max_retries"]):
        try:
            success = generate_image(client, full_prompt, output_filename, aspect_ratio)
            if success:
                print(f"  ✓ Saved to {output_filename}")
                return True
            else:
                raise Exception("Generation returned no valid image")
        except Exception as e:
            wait_time = (2 ** attempt) * 2
            print(f"  ✗ Attempt {attempt + 1}/{CONFIG['max_retries']} failed: {e}")
            if attempt < CONFIG["max_retries"] - 1:
                print(f"  ⏳ Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"  ✗ Failed after {CONFIG['max_retries']} attempts")
                return False
    return False

def validate_prompts_json(data):
    """Validate the structure of prompts.json"""
    required_keys = ["projects", "meta_upgrades", "upgrades", "jobs"]
    valid = True
    
    for key in required_keys:
        if key not in data:
            print(f"⚠ Warning: '{key}' missing from prompts.json")
            valid = False
        elif not isinstance(data[key], list):
            print(f"✗ Error: '{key}' must be a list")
            valid = False
        else:
            # Validate each item has required fields
            for i, item in enumerate(data[key]):
                if key == "jobs":
                    if "title" not in item or "desc" not in item:
                        print(f"✗ Error: Item {i} in '{key}' missing 'title' or 'desc'")
                        valid = False
                else:
                    if "id" not in item or "desc" not in item:
                        print(f"✗ Error: Item {i} in '{key}' missing 'id' or 'desc'")
                        valid = False
    
    return valid

def estimate_cost(total_images):
    """Estimate the cost of generating images"""
    # Approximate pricing for Gemini image generation
    cost_per_image = 0.04  # Estimated cost per image
    estimated = total_images * cost_per_image
    print(f"\n💰 Estimated cost: ${estimated:.2f} for {total_images} images")
    print(f"   (Based on ~${cost_per_image} per image)")

def process_list(client, category_name, item_list, style_prefix, id_key, file_prefix, ar="1:1"):
    """Process a list of items and generate images"""
    for item in tqdm(item_list, desc=category_name):
        stats["total"] += 1
        
        # Build prompt and filename
        final_prompt = style_prefix + item['desc']
        item_id = item.get(id_key, "unknown").lower().replace(" ", "_")
        filename = os.path.join(OUTPUT_DIR, f"{file_prefix}_{item_id}.png")

        # Skip existing files if configured
        if CONFIG["skip_existing"] and os.path.exists(filename):
            print(f"⊘ Skipping existing: {filename}")
            stats["skipped"] += 1
            continue

        # Dry run mode
        if CONFIG["dry_run"]:
            print(f"[DRY RUN] Would generate: {filename}")
            continue

        # Generate image
        print(f"\n🎨 Generating: {filename}...")
        success = generate_image_with_retry(client, final_prompt, filename, aspect_ratio=ar)
        
        if success:
            stats["success"] += 1
        else:
            stats["failed"] += 1
            stats["failed_items"].append({
                "category": category_name,
                "id": item_id,
                "filename": filename,
                "prompt": final_prompt
            })
        
        # Rate limiting
        if not CONFIG["dry_run"]:
            time.sleep(CONFIG["rate_limit_delay"])

def print_summary():
    """Print generation summary"""
    print("\n" + "="*60)
    print("📊 GENERATION SUMMARY")
    print("="*60)
    print(f"Total items:     {stats['total']}")
    print(f"✓ Successful:    {stats['success']}")
    print(f"⊘ Skipped:       {stats['skipped']}")
    print(f"✗ Failed:        {stats['failed']}")
    
    if stats["failed"] > 0:
        print(f"\n❌ Failed items saved to: failed_generations.json")
        with open("failed_generations.json", "w") as f:
            json.dump(stats["failed_items"], f, indent=2)

def main():
    print("="*60)
    print("🎮 ENTERPRISE CLICKER - AI ASSET GENERATOR")
    print("="*60)
    
    # Create output directory
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        print(f"✓ Created directory: {OUTPUT_DIR}")

    # Load and validate JSON
    try:
        with open('prompts.json', 'r') as f:
            data = json.load(f)
    except FileNotFoundError:
        print("✗ Error: prompts.json not found!")
        return

    print("\n📋 Validating prompts.json...")
    if not validate_prompts_json(data):
        print("✗ Validation failed. Please fix prompts.json")
        return
    print("✓ Validation passed")

    # Count total items
    total_count = sum(len(data.get(cat, [])) for cat in CONFIG["categories"])
    if CONFIG["skip_existing"]:
        existing = sum(1 for cat in CONFIG["categories"] 
                      for item in data.get(cat, [])
                      if os.path.exists(os.path.join(OUTPUT_DIR, 
                          f"{'job' if cat == 'jobs' else cat.rstrip('s')}_{item.get('title' if cat == 'jobs' else 'id', '').lower().replace(' ', '_')}.png")))
        total_count -= existing

    estimate_cost(total_count)

    if CONFIG["dry_run"]:
        print("\n⚠ DRY RUN MODE - No images will be generated")
    
    # Confirm
    if not CONFIG["dry_run"] and total_count > 0:
        response = input(f"\n▶ Generate {total_count} images? (yes/no): ").strip().lower()
        if response != "yes":
            print("❌ Cancelled")
            return

    # Initialize AI client
    client = init_ai() if not CONFIG["dry_run"] else None

    # Process each category
    category_mapping = {
        "projects": (data.get("projects", []), STYLE_ICON, "id", "project", "1:1"),
        "meta_upgrades": (data.get("meta_upgrades", []), STYLE_ICON, "id", "meta", "1:1"),
        "upgrades": (data.get("upgrades", []), STYLE_ICON, "id", "upgrade", "1:1"),
        "jobs": (data.get("jobs", []), STYLE_SPLASH, "title", "job", "16:9")
    }

    for category in CONFIG["categories"]:
        if category in category_mapping:
            items, style, id_key, prefix, ar = category_mapping[category]
            if items:
                print(f"\n{'='*60}")
                print(f"📂 Processing: {category.upper()}")
                print(f"{'='*60}")
                process_list(client, category, items, style, id_key, prefix, ar)

    # Print summary
    print_summary()
    print("\n✓ Done!")


# Enterprise Clicker - AI Asset Generator

This notebook generates pixel art assets for the Enterprise Clicker game using Google's Gemini AI.

## Features
- ✅ Retry logic with exponential backoff
- ✅ Skip existing images
- ✅ Dry run mode for testing
- ✅ Image validation
- ✅ Progress tracking and statistics
- ✅ Cost estimation
- ✅ Failed generation logging

## Configuration

Modify the `CONFIG` dictionary in the code cell below to customize behavior:
- `skip_existing`: Skip files that already exist
- `dry_run`: Preview what will be generated without API calls
- `categories`: Select which categories to generate
- `rate_limit_delay`: Delay between API calls
- `max_retries`: Maximum retry attempts per image
- `validate_images`: Validate generated images

In [ ]:
# Run the main generation process
main()

## Quick Configuration Examples

Run one of these cells before running `main()` to change behavior:

In [ ]:
# Dry run mode - preview without generating
CONFIG["dry_run"] = True
main()

In [ ]:
# Generate only specific categories
CONFIG["categories"] = ["projects", "meta_upgrades"]  # Only these two
CONFIG["dry_run"] = False
main()

In [ ]:
# Regenerate all images (overwrite existing)
CONFIG["skip_existing"] = False
CONFIG["categories"] = ["projects", "meta_upgrades", "upgrades", "jobs"]
main()

## Retry Failed Generations

If some images failed to generate, they're saved to `failed_generations.json`. Use this cell to retry only those:

In [ ]:
# Retry failed generations
def retry_failed():
    try:
        with open("failed_generations.json", "r") as f:
            failed = json.load(f)
        
        if not failed:
            print("✓ No failed items to retry")
            return
        
        print(f"🔄 Retrying {len(failed)} failed generations...")
        client = init_ai()
        
        retried_stats = {"success": 0, "failed": 0}
        
        for item in tqdm(failed, desc="Retrying"):
            print(f"\n🎨 Retrying: {item['filename']}...")
            success = generate_image_with_retry(
                client, 
                item["prompt"], 
                item["filename"],
                "16:9" if "job_" in item["filename"] else "1:1"
            )
            
            if success:
                retried_stats["success"] += 1
            else:
                retried_stats["failed"] += 1
            
            time.sleep(CONFIG["rate_limit_delay"])
        
        print(f"\n📊 Retry Results:")
        print(f"✓ Successful: {retried_stats['success']}")
        print(f"✗ Failed: {retried_stats['failed']}")
        
    except FileNotFoundError:
        print("✗ No failed_generations.json found")

retry_failed()

## Package & Download

Zip all generated images for download:

In [ ]:
# Zip all the generated images
!zip -r generated_images.zip images